In [3]:
import smtplib
from email.mime.image       import MIMEImage
from email.mime.multipart   import MIMEMultipart
from email.mime.text        import MIMEText
from email.mime.message     import MIMEMessage
from email.mime.application import MIMEApplication
from os.path import basename
import email
import os, json
import uuid
import re
import pdb
from bs4 import BeautifulSoup
from CheckEmailHelper import CheckEmailHelper

ImportError: No module named 'CheckEmailHelper'

In [4]:
HOST = 'imap.gmail.com'
directory ="/Users/wxji/Documents/Connect/streamonce-integration-python"

with open(directory + '/env.json') as json_data:  # os.getcwd() Return the current working directory
    env = json.load(json_data)
    apiUrl = env["jive"]["apiBaseUrl"]

with open(directory + '/accounts.json') as json_data:
    account = json.load(json_data)

     
print(account)
# print(file）

{'User5': {'twoStep': '', 'password': '', 'id': '', 'username': '', 'displayName': '', 'email': '', 'alias': '', 'googlePassword': ''}, 'User2': {'twoStep': '', 'password': '', 'id': '', 'username': '', 'displayName': '', 'email': '', 'alias': '', 'googlePassword': ''}, 'jiveAdmin': {'password': '', 'id': '', 'username': '', 'displayName': ''}, 'GoogleAdmin': {'password': '', 'username': ''}, 'User3': {'twoStep': '', 'password': '', 'id': '', 'username': '', 'displayName': '', 'email': '', 'alias': '', 'googlePassword': ''}, 'User4': {'twoStep': '', 'password': '', 'id': '', 'username': '', 'displayName': '', 'email': '', 'alias': '', 'googlePassword': ''}, 'User1': {'twoStep': '', 'password': '', 'id': '', 'username': '', 'displayName': '', 'email': '', 'alias': '', 'googlePassword': ''}}


In [5]:
#origin_Subject="Test Reply Email with image attachment"
origin_Subject="Send Email with image attachment Fri Mar 3 17:11:17 CST 2017"
groupEmail = 'feb-sissy-manualtest <feb-sissy-manualtest@dev.thoughtworks.com>';
# groupEmail = 'CNC3 Tester3 <cnctester3@dev.thoughtworks.com'

Origin = CheckEmailHelper.findEmailBySubject(account["User1"],origin_Subject, groupEmail)

NameError: name 'CheckEmailHelper' is not defined

In [6]:
From=account["User1"]
To=account["User3"]
Cc=groupEmail
Preamble=""
TEXT=""
ImageName = "thumb.jpg"
Images = [{
    "doUpload": True,
    "name": ImageName,
    "path": directory+"/test/asserts/thumb.jpg"
}]

file = open (directory + '/test/example/doc_attach')

HTML="""
<html>
<body>
<div>
<p>Reply from cnctester1 in Email with attachment! </p>
<p>Thank you!</p>
</div>
</body></html>"""

ReplyTo=[]
useAlias = False

FileNotFoundError: [Errno 2] No such file or directory: '/Users/wxji/Documents/Connect/streamonce-integration-python/test/example/doc_attach'

In [7]:
BeautifulSoup('<div>a</div>', 'html.parser').text

'a'

In [8]:
print(Origin.as_string())

NameError: name 'Origin' is not defined

In [9]:
email.charset.add_charset('utf-8', email.charset.SHORTEST, email.charset.QP)
new_text = ""
new_html = ""
#print (TEXT)

if TEXT!="" and HTML!="":
    new_text = TEXT
    new_html = HTML
    print('no')
else :
    import quopri as qp
    origin_html = str(qp.decodestring(Origin.get_payload()[0]._payload[1]._payload), 'utf-8')
    reply_html = HTML
    compressed_origin_html = '<div class="gmail_extra"><div class="gmail_quote"><blockquote class="gmail_quote" style="margin:0 0 0 .8ex;border-left:1px #ccc solid;padding-left:1ex">'
    compressed_origin_html += origin_html + '</blockquote></div></div>'
    new_html = reply_html + compressed_origin_html

    reply_doc = BeautifulSoup(reply_html, 'html.parser')   #BeautifulSoup is to pull data from HTML&XML
    
    origin_doc = BeautifulSoup(origin_html, 'html.parser')
    #拼回复信的所有text
    reply_text = reply_doc.text
    origin_text = re.sub('\n', '\n> ', "\n\n" + origin_doc.text)

    new_text = reply_text + origin_text


print(origin_html)

NameError: name 'Origin' is not defined

In [10]:
new_text

''

In [11]:
text_block = MIMEText(new_text, 'plain', _charset='utf-8')
html_block = MIMEText(new_html, 'html', 'utf-8')

del html_block['Content-Transfer-Encoding']
html_block['Content-Transfer-Encoding'] = 'quoted-printable'

print('Sending mail From: %s , Subject: %s' % (From["email"], origin_Subject))

for xpart in Origin.walk():
    if (xpart.get('Content-Disposition')
        and xpart.get('Content-Disposition').startswith("attachment")):

        xpart.set_type("text/plain")
        xpart.set_payload("Attachment removed: %s (%s, %d bytes)"
                         %(xpart.get_filename(),
                           xpart.get_content_type(),
                           len(xpart.get_payload(decode=True))))
        del xpart["Content-Disposition"]
        del xpart["Content-Transfer-Encoding"]

print(html_block.get_payload())

Sending mail From:  , Subject: Send Email with image attachment Fri Mar 3 17:11:17 CST 2017


NameError: name 'Origin' is not defined

In [12]:
new_block = MIMEMultipart()
new_block["Message-ID"] = email.utils.make_msgid()   #生成新的messageid
new_block["In-Reply-To"] = Origin["Message-ID"]
new_block["References"] = Origin["Message-ID"]
new_block["Subject"] = "Re: "+Origin["Subject"]
new_block["To"] = new_block["Reply-To"] = Origin["Reply-To"] or Origin["From"]
new_block["From"] = From['email']
new_block["Cc"] = groupEmail
# new_block["Delivered-To"] = From["email"]
    
#add attachment
part = MIMEApplication(open(Images[0]["path"],'rb').read(),'image/jpeg') 
part.add_header('Content-Disposition', 'attachment', filename=Images[0]["name"])
del part['Content-Type']
part.add_header('Content-Type','image/jpeg', name=Images[0]["name"])

NameError: name 'Origin' is not defined

In [13]:
multi1 = MIMEMultipart('alternative')
multi1.attach(text_block)
multi1.attach(html_block)

new_block.attach(multi1)
new_block.attach(part)

NameError: name 'part' is not defined

In [14]:
print(new_block.as_string())

Content-Type: multipart/mixed; boundary="===============4323144486227110698=="
MIME-Version: 1.0
Message-ID: <149570558508.19939.17783463233052267298@Sissys-Mac.local>

--===============4323144486227110698==
Content-Type: multipart/alternative; boundary="===============6986415698954643445=="
MIME-Version: 1.0

--===============6986415698954643445==
Content-Type: text/plain; charset="utf-8"
MIME-Version: 1.0
Content-Transfer-Encoding: quoted-printable


--===============6986415698954643445==
Content-Type: text/html; charset="utf-8"
MIME-Version: 1.0
Content-Transfer-Encoding: quoted-printable


--===============6986415698954643445==--

--===============4323144486227110698==--



In [15]:
print

<function print>

In [16]:
print(Origin.as_string())

NameError: name 'Origin' is not defined

In [17]:
server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login(From["email"], From["googlePassword"])
server.send_message(new_block)
server.quit()

print ("success!")

TypeError: Can't convert 'bytes' object to str implicitly

In [ ]:
def filter(txt, oldfile, newfile):
    '''\
    Read a list of names from a file line by line into an output file.
    If a line begins with a particular name, insert a string of text
    after the name before appending the line to the output file.
    '''

    outfile = open(newfile, 'w')
    with open(oldfile, 'r', encoding='utf-8') as infile:
        for line in infile:
            if line.startswith(txt):
                line = line[0:len(txt)] + ' - Truly a great person!\n'
            outfile.write(line)

    outfile.close()
    return # Do I gain anything by including this?

# input the name you want to check against
text = input('Please enter the name of a great person: ')    
letsgo = filter(text,'Spanish', 'Spanish2')

In [18]:
from requests.auth import HTTPBasicAuth
        import requests
        user=account['User1']
  

IndentationError: unexpected indent (<ipython-input-18-a9e388133be0>, line 2)

In [11]:
contentSubject= 'Discussion and reply with attachment from Email2017-03-07T19:51:33.921180'
        url = apiUrl + "/search/contents?filter=search('%s')&filter=subjectonly(true)" % (contentSubject)
        BasicAuth=HTTPBasicAuth(user["username"], user["password"])
        r = requests.get(url, auth=BasicAuth)
        print (r.status_code)
#         assert r.status_code == 200

401


In [14]:
print (url)
        print (r)

https://thoughtworks-preview.jiveon.com/api/core/v3/search/contents?filter=search('Discussion and reply with attachment from Email2017-03-07T19:51:33.921180')&filter=subjectonly(true)
<Response [401]>


In [ ]:
if not jiveHelper.isOnlyResult(r):
            print('findContentBySubject has no result or more than one')
            return None
        return r.json()['list'][0]